In [5]:
pip install tflearn 

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install imgaug

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install ipyplot

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [9]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.saving.save import load_model
from keras.preprocessing.image import img_to_array
import tensorflow_datasets as tfds
from keras.callbacks import EarlyStopping,ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import cv2
import random
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from PIL import Image
import shutil
import json
from keras.applications.vgg16 import VGG16

from keras.applications.resnet import ResNet50
from PIL import *
import PIL.Image

In [10]:

TRAIN_DIR = '/kaggle/input/nn23-sports-image-classification/Train'
TEST_DIR = '/kaggle/input/nn23-sports-image-classification/Test'
train_images=[]
test_images=[]
val_images=[]
IMG_SIZE = 224
num_classes = 6
MODEL_NAME = 'sports_classification-cnn'


In [11]:
def convert_all_image_to_jpg():
    for img in tqdm(os.listdir(TRAIN_DIR)):
        x = img.split(".")
        path = 'drive/MyDrive/data/Train/' + img
        name = 'drive/MyDrive/data/Train/' + x[0]
        im = Image.open(path)
        im.convert('RGB').save(name + ".jpg")
        if (x[1] != 'jpg'):
             os.remove(path) 
#convert_all_image_to_jpg()

In [12]:
    data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        
        data.append(img)
    print(data)
    from sklearn.model_selection import train_test_split
    train,validate=train_test_split(data,random_state=160,test_size=0.2,shuffle=True)
    print(len(train))
    print(len(validate))

100%|██████████| 1681/1681 [00:00<00:00, 1407310.38it/s]

['Yoga_22.jpg', 'Football_119.jpg', 'Swimming_132.jpg', 'Football_182.jpg', 'Yoga_158.jpg', 'Tennis_68.jpg', 'Yoga_416.png', 'Rowing_186.png', 'Yoga_55.jpg', 'Swimming_141.jpg', 'Rowing_42.jpg', 'Swimming_184.png', 'Yoga_30.jpg', 'Tennis_26.jpg', 'Basketball_31.jpg', 'Tennis_93.jpg', 'Rowing_129.jpg', 'Swimming_67.jpg', 'Swimming_112.jpg', 'Swimming_87.jpg', 'Football_96.jpg', 'Tennis_110.jpg', 'Football_138.jpg', 'Basketball_190.png', 'Football_124.jpg', 'Yoga_287.jpg', 'Yoga_198.jpg', 'Tennis_54.jpg', 'Basketball_83.jpg', 'Tennis_67.jpg', 'Rowing_103.jpg', 'Swimming_218.png', 'Basketball_176.png', 'Football_71.jpg', 'Tennis_9.jpg', 'Yoga_413.png', 'Yoga_122.jpg', 'Swimming_233.png', 'Yoga_152.jpg', 'Tennis_156.png', 'Football_350.jpg', 'Rowing_27.jpg', 'Rowing_100.jpg', 'Basketball_30.jpg', 'Tennis_106.jpg', 'Yoga_148.jpg', 'Swimming_129.jpg', 'Yoga_280.jpg', 'Yoga_429.png', 'Basketball_182.png', 'Swimming_180.png', 'Tennis_17.jpg', 'Swimming_234.png', 'Basketball_16.jpg', 'Rowing_15

In [13]:
def augmentation(input_img):
    list_data=[]
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    img = input_img
    i=0
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,)+x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    for batch in datagen.flow(x, batch_size=1):
        list_data.append(img_to_array(batch[0]))
        i += 1
        if i ==2:
            break 


    return list_data

In [14]:
def augmentation_for_balance(input_img):
    list_data=[]
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    img = input_img
    i=0
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,)+x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    for batch in datagen.flow(x, batch_size=1):
        list_data.append(img_to_array(batch[0]))
        i += 1
        if i ==1:
            break 


    return list_data

In [15]:

training_images = []
def balance_train_data():
    basket = 0
    football = 0
    rowing = 0
    swimming = 0
    tennis = 0
    yoga = 0
    for img in tqdm(train):
        imge = img[0:4]
        if imge == 'Bask':
            basket+=1
        elif imge == 'Foot':
            football+=1
        elif imge == 'Rowi':
            rowing+=1
        elif imge == 'Swim':
            swimming+=1
        elif imge == 'Tenn':
            tennis+=1
        elif imge == 'Yoga':
            yoga+=1
    

    f = football
    b = basket
    r = rowing
    s = swimming
    t = tennis
    y = yoga
 
    for img in tqdm(train):
        start = 0
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path,1)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_images.append([img_data,img])

        imge = img[0:4]
        if imge == 'Bask':start = b
        elif imge == 'Foot':start = f
        elif imge == 'Rowi':start = r
        elif imge == 'Swim':start = s
        elif imge == 'Tenn':start = t
        elif imge == 'Yoga':start = y
        while(start < 500):
            for j in train:
                if (j[0:4] != imge):continue
                start+=1
                if (start > 500):break
                path = os.path.join(TRAIN_DIR, j)
                img_data1 = cv2.imread(path,1)
                img_data1 = cv2.cvtColor(img_data1, cv2.COLOR_BGR2RGB)
                img_data1 = cv2.resize(img_data1, (IMG_SIZE, IMG_SIZE))
                aug=augmentation_for_balance(img_data1)
                training_images.append([img_data,j])
               

        if imge == 'Bask':b = 500
        elif imge == 'Foot':f = 500
        elif imge == 'Rowi':r = 500
        elif imge == 'Swim':s = 500
        elif imge == 'Tenn':t = 500
        elif imge == 'Yoga':y = 500
            
balance_train_data()


100%|██████████| 1344/1344 [00:36<00:00, 37.11it/s] 


In [16]:
def count_tarin_data():
    basket = 0
    football = 0
    rowing = 0
    swimming = 0
    tennis = 0
    yoga = 0
    for img in tqdm(training_images):
        imge = img[1][0:4]
        if imge == 'Bask':
            basket+=1
        elif imge == 'Foot':
            football+=1
        elif imge == 'Rowi':
            rowing+=1
        elif imge == 'Swim':
            swimming+=1
        elif imge == 'Tenn':
            tennis+=1
        elif imge == 'Yoga':
            yoga+=1

    print("baske " , basket)
    print("football" , football)
    print("rowing" , rowing)
    print("swimming" , swimming)
    print("tennis" , tennis)
    print("yoga" , yoga)
count_tarin_data()


100%|██████████| 3000/3000 [00:00<00:00, 1303118.48it/s]

baske  500
football 500
rowing 500
swimming 500
tennis 500
yoga 500


In [17]:
def create_label(image_name):
    img = image_name[0:4]
    if img == 'Bask':
        return 0
    elif img == 'Foot':
        return 1
    elif img == 'Rowi':
        return 2
    elif img == 'Swim':
        return 3
    elif img == 'Tenn':
        return 4
    elif img == 'Yoga':
        return 5

In [18]:
def create_train_data():
    training_data = []
  
    for img in tqdm(training_images):
       

        label_list = create_label(img[1])
        training_data.append([np.array(img[0]), label_list])

        #append augmentation to train data
        aug_list = augmentation(img[0]);
        for aug in aug_list:
            training_data.append([np.array(aug), label_list])

    print("Train size = " , len(training_data))
    shuffle(training_data)
    
    return training_data

In [19]:
column1=[]
def create_test_data():
    test_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
      
        path = os.path.join(TEST_DIR, img)
        img_data = cv2.imread(path,1)
        column1.append(img)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        test_data.append([np.array(img_data), 0])

   
    return test_data


In [20]:
def create_validation_data():
    validation_data = []
    t = 0
    for img in tqdm(validate):
        
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path,1)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))

        label_list = create_label(img)
        validation_data.append([np.array(img_data), label_list])

    shuffle(validation_data)
    
    return validation_data

In [21]:

train_images = create_train_data()


X_train = np.array([i[0] for i in train_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_train = np.array([i[1] for i in train_images]).reshape(-1,)
training_images.clear()
train_images.clear()

#==============================================================================================

 
val_images = create_validation_data()

print("val images = ",len(val_images))


X_val = np.array([i[0] for i in val_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_val = np.array([i[1] for i in val_images]).reshape(-1,)

val_images.clear()
#==============================================================================================

 
test_images = create_test_data()
print("test images = ",len(test_images))


X_test = np.array([i[0] for i in test_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_test = np.array([i[1] for i in test_images]).reshape(-1,)
test_images.clear()

100%|██████████| 3000/3000 [01:02<00:00, 48.38it/s]


Train size =  9000


100%|██████████| 337/337 [00:03<00:00, 100.79it/s]


val images =  337


100%|██████████| 688/688 [00:08<00:00, 85.52it/s] 

test images =  688


In [22]:
x=VGG16(weights="imagenet", include_top=False)
x.save_weights('vgg16weight.h5')

2022-12-23 17:54:19.063172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 17:54:19.167989: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 17:54:19.169173: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 17:54:19.172517: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [ ]:

model = Sequential()



model.add(Conv2D(input_shape=(IMG_SIZE,IMG_SIZE,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),name='vgg16'))

model.load_weights('/kaggle/working/vgg16weight.h5')

model.add(Flatten(name='flatten'))

model.add(Dense(256, activation='relu', name='fc1'))

model.add(Dense(128, activation='relu', name='fc2'))


model.add(Dense(6, activation='softmax', name='output'))



model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 0.00005),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

model.fit(X_train, y_train, validation_data = (X_val, Y_val), epochs = 30)



Train on 9000 samples, validate on 337 samples
Epoch 1/30
9000/9000 [==============================] - ETA: 0s - loss: 0.7746 - acc: 0.7872

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


9000/9000 [==============================] - 60s 7ms/sample - loss: 0.7746 - acc: 0.7872 - val_loss: 0.5843 - val_acc: 0.7864
Epoch 2/30
9000/9000 [==============================] - 59s 7ms/sample - loss: 0.3039 - acc: 0.9039 - val_loss: 0.5091 - val_acc: 0.8427
Epoch 3/30
9000/9000 [==============================] - 59s 7ms/sample - loss: 0.1739 - acc: 0.9443 - val_loss: 0.4156 - val_acc: 0.8783
Epoch 4/30
9000/9000 [==============================] - 59s 7ms/sample - loss: 0.1232 - acc: 0.9613 - val_loss: 0.3465 - val_acc: 0.8843
Epoch 5/30
9000/9000 [==============================] - 59s 7ms/sample - loss: 0.0828 - acc: 0.9749 - val_loss: 0.3206 - val_acc: 0.9050
Epoch 6/30
9000/9000 [==============================] - 59s 7ms/sample - loss: 0.0612 - acc: 0.9811 - val_loss: 0.3942 - val_acc: 0.8902
Epoch 7/30
9000/9000 [==============================] - 59s 7ms/sample - loss: 0.0561 - acc: 0.9834 - val_loss: 0.5617 - val_acc: 0.8754
Epoch 8/30
9000/9000 [==============================

In [35]:
model.save_weights('model_weights_V2.h5')

In [34]:
def test():
  Y_test=model.predict(X_test)
  print(len(X_test))
  prediction=[]
  for i in Y_test:
    prediction.append(np.argmax(i))
  data={'image_name':column1,'label':prediction[:688]}
  print(len(column1),len(prediction))
  df=pd.DataFrame(data)
  df.to_csv('VGG16_epoch.csv',index=False)
  print(df)

test()

688
688 688
    image_name  label
0      623.jpg      2
1      208.jpg      5
2      473.jpg      0
3      333.jpg      1
4      537.jpg      1
..         ...    ...
683    364.jpg      3
684     90.jpg      3
685    599.jpg      4
686     25.jpg      4
687    147.jpg      3

[688 rows x 2 columns]
